In [3]:
import pandas as pd
import pandas_profiling as pp
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pp.clear_config()

In [4]:
data = pd.read_csv("/mnt/c/Course Data/Medicare_Provider_Util_Payment_PUF_CY2018/Medicare_Provider_Util_Payment_PUF_CY2018.txt", sep='\t')

/home/barryz/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
#checking the amount of null data in each collumn
data.isnull().sum()

npi                                       0
nppes_provider_last_org_name            144
nppes_provider_first_name            419699
nppes_provider_mi                   3001419
nppes_credentials                    732427
nppes_provider_gender                419599
nppes_entity_code                         1
nppes_provider_street1                 1688
nppes_provider_street2              6265699
nppes_provider_city                    1687
nppes_provider_zip                     1688
nppes_provider_state                      1
nppes_provider_country                    1
provider_type                             1
medicare_participation_indicator          1
place_of_service                          1
hcpcs_code                                1
hcpcs_description                         1
hcpcs_drug_indicator                      1
line_srvc_cnt                             1
bene_unique_cnt                           1
bene_day_srvc_cnt                         1
average_Medicare_allowed_amt    

In [6]:
data.provider_type.value_counts()

Diagnostic Radiology                                       1246653
Internal Medicine                                          1054533
Family Practice                                             952894
Nurse Practitioner                                          630826
Cardiology                                                  431235
Physician Assistant                                         407751
Orthopedic Surgery                                          295567
Physical Therapist in Private Practice                      293847
Anesthesiology                                              244192
Ophthalmology                                               239707
Emergency Medicine                                          230742
Dermatology                                                 206238
Podiatry                                                    189250
Urology                                                     182574
Gastroenterology                                            18

In [7]:
data.describe()

,npi,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
count,9.961866e+06,9.961865e+06,9.961865e+06,9.961865e+06,9.961865e+06,9.961865e+06,9.961865e+06,9.961865e+06
mean,1.499541e+09,2.493754e+02,8.734895e+01,1.393506e+02,1.031440e+02,3.644679e+02,7.885533e+01,7.930758e+01
std,2.877158e+08,6.102059e+03,1.211845e+03,2.046794e+03,2.938051e+02,1.114721e+03,2.284682e+02,2.289130e+02
min,1.000000e+00,4.500000e+00,1.100000e+01,1.100000e+01,1.666667e-04,1.666667e-04,0.000000e+00,0.000000e+00
25%,1.245730e+09,2.000000e+01,1.700000e+01,2.000000e+01,2.279000e+01,6.000000e+01,1.909387e+01,1.980464e+01
50%,1.497959e+09,4.300000e+01,3.200000e+01,4.000000e+01,6.606000e+01,1.500000e+02,4.823550e+01,4.899765e+01
75%,1.740646e+09,1.170000e+02,7.500000e+01,1.060000e+02,1.140738e+02,3.060000e+02,8.599000e+01,8.560725e+01
max,1.993000e+09,1.076244e+07,8.898900e+05,1.731455e+06,5.528929e+04,9.999999e+04,4.334680e+04,4.334680e+04


In [17]:
# quick cleaning
# Only take individual Cardiologist, providers who practice in the US, participates in Medicare
data = data[1:]
data = data[data.medicare_participation_indicator == 'Y']
data = data[data.nppes_entity_code == 'I']
data = data[data.nppes_provider_country == 'US']
data = data[(data.provider_type == 'Cardiology') | (data.provider_type == 'Pulmonary Disease') | (data.provider_type == 'Nephrology')]

In [18]:
data.to_csv('COVID doctors.csv')

### EDA:
* Feature Summary and Type
    * Type
    * Histogram
    
* Correlation among the numeric features
* missing data visualization

In [21]:
list(data.columns)

['npi',
 'nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_entity_code',
 'nppes_provider_street1',
 'nppes_provider_street2',
 'nppes_provider_city',
 'nppes_provider_zip',
 'nppes_provider_state',
 'nppes_provider_country',
 'provider_type',
 'medicare_participation_indicator',
 'place_of_service',
 'hcpcs_code',
 'hcpcs_description',
 'hcpcs_drug_indicator',
 'line_srvc_cnt',
 'bene_unique_cnt',
 'bene_day_srvc_cnt',
 'average_Medicare_allowed_amt',
 'average_submitted_chrg_amt',
 'average_Medicare_payment_amt',
 'average_Medicare_standard_amt']

In [24]:
data.hcpcs_code.value_counts()

99214    38255
99232    27665
99213    27184
93306    25873
99204    24190
         ...  
00520        1
92611        1
86301        1
33530        1
33224        1
Name: hcpcs_code, Length: 1467, dtype: int64

In [25]:
EDA_data = data[[
 'nppes_provider_gender',
 'nppes_provider_city',
 'nppes_provider_zip',
 'nppes_provider_state',
 'provider_type',
 'place_of_service',
 'line_srvc_cnt',
 'bene_unique_cnt',
 'bene_day_srvc_cnt',
 'average_Medicare_allowed_amt',
 'average_submitted_chrg_amt',
 'average_Medicare_payment_amt',
 'average_Medicare_standard_amt']]

In [26]:
profile = pp.ProfileReport(EDA_data, correlations={
         "kendall": {"calculate": False},
         "phi_k": {"calculate": False},
         "cramers": {"calculate": False}}
                          )
profile.to_file('EDA.html')

In [27]:
data.provider_type.describe()

count         683408
unique             3
top       Cardiology
freq          431204
Name: provider_type, dtype: object